In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import yfinance as yf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Libraries imported successfully")

## 2. Load and Explore Data

In [ ]:
# Fetch gold price data
gold = yf.download('GLD', start='2015-01-01', end='2026-01-18', progress=False)
gold['Price'] = (gold['Close'] * 10.8).round(0)  # Convert to approximate gold price per ounce
gold = gold[['Price']].copy()

print(f"Gold data shape: {gold.shape}")
print(f"Date range: {gold.index.min()} to {gold.index.max()}")
print(f"\nData summary:")
print(gold['Price'].describe())
print(f"\nMissing values: {gold['Price'].isna().sum()}")

## 3. Perform Seasonal Decomposition

In [ ]:
# Seasonal decomposition (365-day seasonal period for annual seasonality)
decomposition = seasonal_decompose(gold['Price'], model='additive', period=365, )

# Plot decomposition
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=('Original Series', 'Trend', 'Seasonal', 'Residual'),
    specs=[[{'secondary_y': False}], [{'secondary_y': False}], [{'secondary_y': False}], [{'secondary_y': False}]],
    vertical_spacing=0.08
)

# Original
fig.add_trace(
    go.Scatter(x=gold.index, y=gold['Price'], mode='lines', name='Original', line=dict(color='blue')),
    row=1, col=1
)

# Trend
fig.add_trace(
    go.Scatter(x=decomposition.trend.index, y=decomposition.trend, mode='lines', name='Trend', line=dict(color='red')),
    row=2, col=1
)

# Seasonal
fig.add_trace(
    go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, mode='lines', name='Seasonal', line=dict(color='green')),
    row=3, col=1
)

# Residual
fig.add_trace(
    go.Scatter(x=decomposition.resid.index, y=decomposition.resid, mode='lines', name='Residual', line=dict(color='purple')),
    row=4, col=1
)

fig.update_xaxes(title_text='Date', row=4, col=1)
fig.update_layout(height=900, showlegend=True, title_text='Seasonal Decomposition of Gold Prices')
fig.show()

print("✓ Seasonal decomposition visualization complete")

## 4. Prepare Data for Modeling - Monthly Aggregation

In [ ]:
# Aggregate to monthly data for better seasonal pattern
gold_monthly = gold['Price'].resample('MS').mean()

# 80-20 train-test split
train_size = int(len(gold_monthly) * 0.8)
train = gold_monthly.iloc[:train_size]
test = gold_monthly.iloc[train_size:]

print(f"Monthly data shape: {gold_monthly.shape}")
print(f"Training set: {len(train)} observations ({len(train)/len(gold_monthly)*100:.1f}%)")
print(f"Test set: {len(test)} observations ({len(test)/len(gold_monthly)*100:.1f}%)")
print(f"\nTrain period: {train.index.min()} to {train.index.max()}")
print(f"Test period: {test.index.min()} to {test.index.max()}")

## 5. Fit Additive Holt-Winters Model

In [ ]:
# Fit additive model (seasonality is additive)
model_add = ExponentialSmoothing(
    train,
    seasonal_periods=12,
    trend='add',
    seasonal='add',
    initialization_method='estimated'
)
fit_add = model_add.fit(optimized=True)

print("Additive Holt-Winters Model Summary:")
print(fit_add.summary())

## 6. Extract Additive Model Parameters

In [ ]:
# Extract parameters for additive model
alpha_add = fit_add.params['smoothing_level']
beta_add = fit_add.params['smoothing_trend']
gamma_add = fit_add.params['smoothing_seasonal']

print(f"Additive Model Parameters:")
print(f"  Alpha (level smoothing): {alpha_add:.4f}")
print(f"  Beta (trend smoothing): {beta_add:.4f}")
print(f"  Gamma (seasonal smoothing): {gamma_add:.4f}")

print(f"\nInterpretation (Additive):")
print(f"  - Level: {alpha_add*100:.1f}% weight on current observation")
print(f"  - Trend: {beta_add*100:.1f}% weight on recent trend change")
print(f"  - Seasonal: {gamma_add*100:.1f}% weight on recent seasonal pattern")

# Extract final components
level_add = fit_add.level.iloc[-1]
trend_add = fit_add.trend.iloc[-1]

print(f"\nFinal Components (Additive):")
print(f"  Level: {level_add:.4f}")
print(f"  Trend: {trend_add:.6f}")

## 7. Fit Multiplicative Holt-Winters Model

In [ ]:
# Fit multiplicative model (seasonality is multiplicative)
model_mul = ExponentialSmoothing(
    train,
    seasonal_periods=12,
    trend='add',
    seasonal='mul',
    initialization_method='estimated'
)
fit_mul = model_mul.fit(optimized=True)

print("Multiplicative Holt-Winters Model Summary:")
print(fit_mul.summary())

## 8. Extract Multiplicative Model Parameters

In [ ]:
# Extract parameters for multiplicative model
alpha_mul = fit_mul.params['smoothing_level']
beta_mul = fit_mul.params['smoothing_trend']
gamma_mul = fit_mul.params['smoothing_seasonal']

print(f"Multiplicative Model Parameters:")
print(f"  Alpha (level smoothing): {alpha_mul:.4f}")
print(f"  Beta (trend smoothing): {beta_mul:.4f}")
print(f"  Gamma (seasonal smoothing): {gamma_mul:.4f}")

print(f"\nInterpretation (Multiplicative):")
print(f"  - Level: {alpha_mul*100:.1f}% weight on current observation")
print(f"  - Trend: {beta_mul*100:.1f}% weight on recent trend change")
print(f"  - Seasonal: {gamma_mul*100:.1f}% weight on recent seasonal pattern")

# Extract final components
level_mul = fit_mul.level.iloc[-1]
trend_mul = fit_mul.trend.iloc[-1]

print(f"\nFinal Components (Multiplicative):")
print(f"  Level: {level_mul:.4f}")
print(f"  Trend: {trend_mul:.6f}")

## 9. Generate Forecasts from Both Models

In [ ]:
# Generate forecasts
forecast_add = pd.Series(fit_add.forecast(steps=len(test)).values, index=test.index)
forecast_mul = pd.Series(fit_mul.forecast(steps=len(test)).values, index=test.index)

print(f"Forecast horizon: {len(test)} months")
print(f"Forecast period: {test.index[0].strftime('%Y-%m')} to {test.index[-1].strftime('%Y-%m')}")
print(f"\nAdditive Model - First 3 forecasts: {forecast_add.iloc[:3].values}")
print(f"Additive Model - Last 3 forecasts: {forecast_add.iloc[-3:].values}")
print(f"\nMultiplicative Model - First 3 forecasts: {forecast_mul.iloc[:3].values}")
print(f"Multiplicative Model - Last 3 forecasts: {forecast_mul.iloc[-3:].values}")

## 10. Evaluate and Compare Models

In [ ]:
# Calculate performance metrics for both models
def evaluate_model(test, forecast, train, model_name):
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test, forecast)
    mape = np.mean(np.abs((test - forecast) / test)) * 100
    
    # Naive seasonal baseline (use value from 12 months ago)
    naive_seasonal = pd.Series([train.iloc[-(len(test)-i)] for i in range(len(test))], index=test.index)
    naive_rmse = np.sqrt(mean_squared_error(test, naive_seasonal))
    improvement = ((naive_rmse - rmse) / naive_rmse * 100)
    
    print(f"\n{model_name} Performance:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    print(f"  Naive Seasonal RMSE: {naive_rmse:.4f}")
    print(f"  Improvement over Naive: {improvement:.2f}%")
    
    return {'rmse': rmse, 'mae': mae, 'mape': mape, 'improvement': improvement}

print("="*70)
print("MODEL COMPARISON")
print("="*70)

metrics_add = evaluate_model(test, forecast_add, train, "Additive Model")
metrics_mul = evaluate_model(test, forecast_mul, train, "Multiplicative Model")

print(f"\nBetter Model: {'Additive' if metrics_add['rmse'] < metrics_mul['rmse'] else 'Multiplicative'}")
print(f"RMSE Difference: {abs(metrics_add['rmse'] - metrics_mul['rmse']):.4f}")

## 11. Visualize Additive vs Multiplicative Forecasts

In [ ]:
# Create comparison plot
fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(
    x=train.index, y=train.values,
    mode='lines',
    name='Training Data',
    line=dict(color='blue', width=1)
))

# Test data (actual)
fig.add_trace(go.Scatter(
    x=test.index, y=test.values,
    mode='lines',
    name='Test Data (Actual)',
    line=dict(color='green', width=2)
))

# Additive forecast
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_add.values,
    mode='lines',
    name='Additive Forecast',
    line=dict(color='red', width=2, dash='dash')
))

# Multiplicative forecast
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_mul.values,
    mode='lines',
    name='Multiplicative Forecast',
    line=dict(color='orange', width=2, dash='dot')
))

fig.update_layout(
    title="Holt-Winters: Additive vs Multiplicative Forecasts",
    xaxis_title='Date',
    yaxis_title='Price ($)',
    hovermode='x unified',
    template='plotly_white',
    height=500
)
fig.show()

print("✓ Forecast comparison visualization complete")

## 12. Visualize Model Components

In [ ]:
# Visualize additive components
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Level (Additive)', 'Trend (Additive)', 'Seasonal (Additive)', 'Actual vs Fitted (Additive)'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]],
    vertical_spacing=0.12
)

# Level
fig.add_trace(
    go.Scatter(x=train.index, y=fit_add.level, mode='lines', name='Level', line=dict(color='blue')),
    row=1, col=1
)

# Trend
fig.add_trace(
    go.Scatter(x=train.index, y=fit_add.trend, mode='lines', name='Trend', line=dict(color='red')),
    row=1, col=2
)
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=1, col=2)

# Seasonal
fig.add_trace(
    go.Scatter(x=train.index, y=fit_add.seasonal, mode='lines', name='Seasonal', line=dict(color='green')),
    row=2, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=2, col=1)

# Actual vs Fitted
fig.add_trace(
    go.Scatter(x=train.index, y=train.values, mode='lines', name='Actual', line=dict(color='blue')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=train.index, y=fit_add.fittedvalues, mode='lines', name='Fitted', line=dict(color='red', dash='dash')),
    row=2, col=2
)

fig.update_layout(height=700, showlegend=True, title_text='Additive Model Components')
fig.show()

print("✓ Additive components visualization complete")

## 13. Residual Analysis

In [ ]:
# Calculate residuals
residuals_add = test - forecast_add
residuals_mul = test - forecast_mul

print("Residual Statistics (Additive Model):")
print(f"  Mean: {residuals_add.mean():.4f} (should be ~0)")
print(f"  Std Dev: {residuals_add.std():.4f}")
print(f"  Min: {residuals_add.min():.4f}")
print(f"  Max: {residuals_add.max():.4f}")

print("\nResidual Statistics (Multiplicative Model):")
print(f"  Mean: {residuals_mul.mean():.4f} (should be ~0)")
print(f"  Std Dev: {residuals_mul.std():.4f}")
print(f"  Min: {residuals_mul.min():.4f}")
print(f"  Max: {residuals_mul.max():.4f}")

# Create residual diagnostics plot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Residuals Over Time (Add)', 'Residuals Over Time (Mul)', 'Histogram (Add)', 'Histogram (Mul)'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

# Additive residuals time series
fig.add_trace(
    go.Scatter(x=test.index, y=residuals_add, mode='lines+markers', name='Residuals (Add)'),
    row=1, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=1)

# Multiplicative residuals time series
fig.add_trace(
    go.Scatter(x=test.index, y=residuals_mul, mode='lines+markers', name='Residuals (Mul)', line=dict(color='orange')),
    row=1, col=2
)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=2)

# Histograms
fig.add_trace(
    go.Histogram(x=residuals_add, name='Distribution (Add)', nbinsx=15),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(x=residuals_mul, name='Distribution (Mul)', nbinsx=15),
    row=2, col=2
)

fig.update_layout(height=700, showlegend=True, title_text="Residual Diagnostics")
fig.show()

print("\n✓ Residual analysis complete")

## 14. Key Insights and Summary

In [ ]:
print("="*70)
print("KEY INSIGHTS: HOLT-WINTERS' SEASONAL METHOD")
print("="*70)

print(f"\nOptimal Parameters (Additive):")
print(f"  α (level smoothing): {alpha_add:.4f}")
print(f"  β (trend smoothing): {beta_add:.4f}")
print(f"  γ (seasonal smoothing): {gamma_add:.4f}")

print(f"\nOptimal Parameters (Multiplicative):")
print(f"  α (level smoothing): {alpha_mul:.4f}")
print(f"  β (trend smoothing): {beta_mul:.4f}")
print(f"  γ (seasonal smoothing): {gamma_mul:.4f}")

print(f"\nAdditive vs Multiplicative:")
print(f"  Additive RMSE: {metrics_add['rmse']:.4f}")
print(f"  Multiplicative RMSE: {metrics_mul['rmse']:.4f}")
print(f"  Better Model: {'Additive' if metrics_add['rmse'] < metrics_mul['rmse'] else 'Multiplicative'}")

print(f"\nWhen to Use Additive:")
print(f"  - Seasonal variation is roughly constant over time")
print(f"  - Absolute seasonal deviation doesn't depend on level")
print(f"  - Formula: ŷ(t+h) = ℓ(t) + h·b(t) + s(t-m+h)")

print(f"\nWhen to Use Multiplicative:")
print(f"  - Seasonal variation increases/decreases with level")
print(f"  - Percentage seasonal deviation is roughly constant")
print(f"  - Formula: ŷ(t+h) = (ℓ(t) + h·b(t)) × s(t-m+h)")

print(f"\nHolt-Winters Three Components:")
print(f"  1. Level: The base level of the series")
print(f"  2. Trend: The linear upward/downward movement")
print(f"  3. Seasonal: The repeating pattern (12-month cycle)")

print(f"\nExponential Smoothing Progression:")
print(f"  Day 9: SES (Level only) → Flat forecasts")
print(f"  Day 10: Holt's (Level + Trend) → Trending forecasts")
print(f"  Day 11: Holt-Winters (Level + Trend + Seasonal) ← YOU ARE HERE")
print(f"         → Trending + Seasonal forecasts")

print("\n" + "="*70)
print("✓ Analysis complete! Exponential smoothing family covered.")
print("="*70)